In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import Word2Vec
from pyspark.ml.recommendation import ALS
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import *

# https://stackoverflow.com/questions/57451719/since-spark-2-3-the-queries-from-raw-json-csv-files-are-disallowed-when-the-ref
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

FILE_NAME = "tweets.json"

original_data = spark.read.option("multiline","true").json(FILE_NAME).cache()

selectId = "480875170"

# Q1

# find all replies
user_replyTo = original_data.select("user_id", "replyto_id").where("replyto_id is not null")
# find all retweets
user_retweet = original_data.select("user_id", "retweet_id").where("retweet_id is not null")

# combine two data:(user_id, relatedTweetId)
user_combine = user_retweet.union(user_replyTo)
user_data = user_combine.rdd.map(lambda row:(row[0], str(row[1]))).groupByKey().mapValues(list)

def changeToString(row):
    user_id = row[0]
    data = " ".join(row[1])
    return (user_id, data)
    
    
user_tweet_string = user_data.map(changeToString)

train_data = user_tweet_string.toDF(["user_id", "data"])

# idf train data
# https://spark.apache.org/docs/latest/ml-features.html#tf-idf
tokenizer = Tokenizer(inputCol="data", outputCol="words")
wordsData = tokenizer.transform(train_data)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures",numFeatures=131072)
featurizedData = hashingTF.transform(wordsData)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

# drop useless columns
clearData = rescaledData.drop("data").drop("words").drop("rawFeatures")

target_user = clearData.filter("user_id =" + selectId).collect()[0][1]
other_user = clearData.filter("user_id !=" + selectId).rdd.map(lambda row : (row[0], row[1]))

def cosine(a,b):
    return a.dot(b) / (a.norm(2) * b.norm(2))

found_user = other_user.map(lambda row:(row[0], cosine(target_user, row[1])))
print("Top five users with similar interest based on TF-IDF:")
print(found_user.sortBy(lambda row:row[1], ascending=False).take(5))


# Word2Vec
text_data = user_tweet_string.toDF(["user_id", "text"])
word_data = text_data.select('user_id', split("text",' ').alias("words"))
# word_data.show(100, truncate=False)

# train data
word2Vec = Word2Vec(vectorSize=300, minCount=1, inputCol="words", outputCol="vector")
model = word2Vec.fit(word_data)
result = model.transform(word_data).drop("words")

target_user_1 = result.filter("user_id =" + selectId).collect()[0][1]
other_user_1 = result.filter("user_id !=" + selectId).rdd.map(lambda row : (row[0], row[1]))
found_user_1 = other_user_1.map(lambda row:(row[0], cosine(target_user_1, row[1])))

print("Top five users with similar interest based on Word2Vec:")
print(found_user_1.sortBy(lambda row:row[1], ascending=False).take(5))


# get user_id 
user_ids = original_data.select("user_id").rdd.map(lambda row:row[0]).distinct().collect()

# create dictionary for user_id
user_id_mapper = dict()
for user_id in user_ids:
    user_id_mapper[user_id] = len(user_id_mapper)

def extract_id(rows):
    return [(mention_user[0]) for mention_user in rows[0]]

# extract mentioned user id
mention_user_info = original_data.select("user_mentions").where("user_mentions is not null")
mention_user_ids = mention_user_info.rdd.flatMap(extract_id).distinct().collect()
# create dictionary for mentioned user id
mention_user_id_mapper = dict()
for mention_user_id in mention_user_ids:
    mention_user_id_mapper[mention_user_id] = len(mention_user_id_mapper)

# sc.broadcast(user_id_mapper)
# sc.broadcast(mention_user_id_mapper)

def extract_data(rows):
    user_id = rows[0]
    return [(user_id, mention_user_id_mapper[mention_user[0]], 1) for mention_user in rows[1]]

def create_key(rows):
    user_id = rows[0] 
    mention_user = rows[1] 
    counter = rows[2]
    return ((user_id,mention_user), counter)

mention_user = original_data.select("user_id","user_mentions").where("user_mentions is not null")
# extract data
clear_data = mention_user.rdd.map(lambda row:(user_id_mapper[row[0]],row[1])).flatMap(extract_data)
# combine user_id mention_user_id as a key
key_data = clear_data.map(create_key)
# calculate                    
clear_key = key_data.reduceByKey(lambda a,b : a + b).map(lambda row : (row[0][0], row[0][1], row[1]))
train_data = clear_key.toDF(["user_id", "mention_user_id", "counter"])


# indexer_1 = StringIndexer(inputCol="user_id", outputCol="user_id_Index")
# indexed_1 = indexer_1.fit(train_data).transform(train_data)

# indexer_2 = StringIndexer(inputCol="mention_user_id", outputCol="mention_user_id_Index")
# indexed_2 = indexer_2.fit(indexed_1).transform(indexed_1)

# a_b = indexed_2.drop("user_id").drop("mention_user_id")
# re_format = a_b[["user_id_Index","mention_user_id_Index","counter"]]

# https://spark.apache.org/docs/latest/ml-collaborative-filtering.html
# train data
als = ALS(userCol="user_id", itemCol="mention_user_id", ratingCol="counter",coldStartStrategy="drop")
model = als.fit(train_data)
model_recommend = model.recommendForAllUsers(5).collect()

# converter = IndexToString(inputCol="user_id_Index", outputCol="raw_user_id_Index")
# converted = converter.transform(model_recommend)
# converted.show()

print("user recomendation is:")
for rows in model_recommend:
    print(user_ids[rows[0]], ":", end=" ")
    for data in rows[1]:
        print(mention_user_ids[data[0]], end="   ")
    print()

sc.stop()
# for rows in model_recommend:
#     if user_ids[rows[0]] == 1242709697397952512:
#         print(user_ids[rows[0]], ":", end=" ")
#         for data in rows[1]:
#             print(mention_user_ids[data[0]], end="   ")


Top five users with similar interest based on TF-IDF:
[(1012137970885906432, 0.43845261466356117), (282643059, 0.4010760488191615), (138309569, 0.39620107225594015), (545185861, 0.38650057152931333), (848329022211620864, 0.3659575900891094)]
Top five users with similar interest based on Word2Vec:
[(1323090359581200385, 0.929142498038134), (875668452463259648, 0.9005889537735137), (1263371505733218304, 0.9005889537735137), (581812461, 0.9005889537735137), (546664702, 0.887339978176648)]
user recomendation is:
17375281 : 111404710   23424533   24259259   61521038   3240396234   
56722736 : 24259259   346102488   1244013067790364673   3094649957   17674244   
866949673461981184 : 24259259   2374745089   55060090   21406834   590120366   
776625931234390017 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1220079661201399809 : 380648579   32828872   100248020   1656958189   14885540   
107548394 : 9814812   3240396234   26574283   1891266440   185025785   
14849092 :

1709590752 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1282841766362861568 : 9814812   3240396234   26574283   1891266440   185025785   
3094649957 : 24259259   3094649957   55060090   1189707154208546816   1094021282503352320   
30874049 : 24259259   346102488   1244013067790364673   3094649957   17674244   
3154773352 : 133081348   408793153   1267708379138535424   9814812   1155838084522618881   
1671585252 : 24259259   3094649957   346102488   1244013067790364673   17674244   
1571268493 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1180372774822535168 : 24259259   346102488   1244013067790364673   3094649957   17674244   
16332783 : 9814812   3240396234   26574283   1891266440   185025785   
24900554 : 24259259   346102488   1244013067790364673   3094649957   17674244   
222349887 : 24259259   346102488   1244013067790364673   3094649957   17674244   
176929421 : 9814812   3240396234   26574283   1891266440   185025785   
12708

1241006463528640513 : 133081348   408793153   1267708379138535424   9814812   1155838084522618881   
53179411 : 24259259   3094649957   346102488   1244013067790364673   17674244   
2823341573 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1122790373372284928 : 56488059   20457806   169963553   894989891632861184   250435513   
26758508 : 45565185   348687298   1373485399   2838696137   242349332   
1538511265 : 24259259   346102488   1244013067790364673   3094649957   17674244   
965251668072779776 : 24259259   13393052   3240396234   1327359720   1015241814   
2539547052 : 24259259   3094649957   346102488   1244013067790364673   17674244   
773633195594178561 : 9814812   3240396234   26574283   1891266440   185025785   
765071010 : 94482117   24259259   1943139204   3240396234   9814812   
1136317807954731014 : 9814812   3240396234   26574283   1891266440   185025785   
293260667 : 24259259   346102488   1244013067790364673   3094649957   17674244   
9371241

1057118005652467712 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1286301543193382912 : 24259259   346102488   3094649957   1244013067790364673   17674244   
4849881125 : 1230064814589915136   370219796   738767160395321345   950322380412342273   14434063   
3902545181 : 24259259   346102488   1244013067790364673   3094649957   17674244   
2163447613 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1400022122 : 24259259   346102488   1244013067790364673   3094649957   17674244   
151497921 : 114968487   157981564   1321935792416149505   1242082182748872705   59533564   
118557374 : 24259259   3094649957   346102488   1244013067790364673   17674244   
1358865992277917696 : 20457806   56488059   939091   17490707   1110032180237852672   
452628476 : 24259259   13393052   3240396234   1327359720   1015241814   
54560396 : 23085502   449027230   24259259   1348750216225296384   1153669884510920704   
437532431 : 9814812   3240396234   265742

1267255422798442496 : 20457806   56488059   1349149096909668363   804414007377416192   3737424859   
1250953106742886400 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1153952256305324032 : 83440337   260371201   860654500431880192   1024822351609901062   180887271   
1296037183 : 24259259   346102488   1244013067790364673   3094649957   17674244   
367300545 : 380648579   32828872   100248020   1656958189   14885540   
54999242 : 191807697   1599212660   221718444   1652541   237341484   
115978171 : 9814812   3240396234   26574283   1891266440   185025785   
3156951007 : 24259259   17674244   153944899   3094649957   346102488   
2219820115 : 24259259   94482117   2195885540   1943139204   3240396234   
1093587784621797378 : 191807697   1599212660   221718444   1652541   237341484   
594770658 : 370219796   950322380412342273   738767160395321345   346102488   1244013067790364673   
25576161 : 1012351320953708545   14335586   41704988   17112878   83645066584

...

1310021689 : 20457806   56488059   192143994   16129920   250917489   
1136215915580854272 : 24259259   346102488   1244013067790364673   3094649957   17674244   
63071049 : 24259259   94482117   2195885540   1943139204   3240396234   
789832199394713601 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1177486338 : 24259259   346102488   1244013067790364673   3094649957   17674244   
790729873 : 9814812   3240396234   94482117   24259259   15115280   
3670308194 : 24259259   3094649957   346102488   1244013067790364673   17674244   
66692395 : 2361630415   3152555541   757303975   2987156301   15913671   
471226357 : 346102488   1244013067790364673   24259259   9814812   17674244   
132016606 : 9814812   3240396234   94482117   24259259   15115280   
14632610 : 133081348   408793153   1267708379138535424   9814812   1155838084522618881   
1726988131 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1070015196633063424 : 9814812   299273962  

62002793 : 9814812   3240396234   26574283   1891266440   185025785   
31067620 : 3290364847   837335683215798272   32840323   132630502   14296273   
490903655 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1278090435039244296 : 380648579   32828872   100248020   1656958189   14885540   
48698695 : 9814812   3240396234   26574283   1891266440   185025785   
370907274 : 9814812   3240396234   26574283   1891266440   185025785   
251910252 : 56488059   20457806   1349149096909668363   15781769   21705181   
91191876 : 24259259   13393052   3240396234   1327359720   1015241814   
192261864 : 9814812   3240396234   26574283   1891266440   185025785   
1901724067 : 24259259   346102488   1244013067790364673   3094649957   17674244   
2381079758 : 147618542   17445167   774301541939412992   4162785695   842678603305377793   
919389403453079553 : 54954921   102754598   9300262   9814812   2729061   
588484683 : 4970411   380648579   346102488   100248020   32828872  

17869506 : 9814812   3240396234   94482117   24259259   15115280   
775128877123403777 : 24259259   346102488   1244013067790364673   3094649957   17674244   
708898201969897473 : 133081348   408793153   1267708379138535424   9814812   1155838084522618881   
327083666 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1308023148940582918 : 9814812   3240396234   26574283   1891266440   185025785   
1688117036 : 17674244   1037321378   24259259   346102488   1244013067790364673   
1213244702 : 9814812   3240396234   26574283   1891266440   185025785   
1302952578 : 9814812   3240396234   26574283   1891266440   185025785   
140892986 : 9814812   3240396234   26574283   1891266440   185025785   
3003440777 : 380648579   32828872   100248020   1656958189   14885540   
2634111053 : 24259259   346102488   1244013067790364673   3094649957   17674244   
843606578 : 18831926   15766082   335060282   122392913   1015241814   
17490707 : 56488059   20457806   939091   134914

187940550 : 9814812   3240396234   26574283   1891266440   185025785   
2571063955 : 9814812   3240396234   26574283   1891266440   185025785   
1251717190186868741 : 24259259   3094649957   346102488   1244013067790364673   17674244   
950235235 : 133081348   408793153   1267708379138535424   9814812   1155838084522618881   
1071887331001581573 : 24259259   346102488   1244013067790364673   3094649957   17674244   
224604544 : 9814812   3240396234   26574283   1891266440   185025785   
1240850363005038592 : 20457806   56488059   1349149096909668363   57073389   77509999   
3009063932 : 24259259   346102488   1244013067790364673   3094649957   17674244   
3228029808 : 24259259   3094649957   346102488   1244013067790364673   17674244   
85181501 : 24259259   346102488   1244013067790364673   3094649957   17674244   
134551745 : 24259259   346102488   1244013067790364673   3094649957   17674244   
1311983978350407681 : 9814812   3240396234   26574283   1891266440   185025785   
43895327

2280035060 : 9814812   3240396234   26574283   1891266440   185025785   
52592089 : 24259259   346102488   1244013067790364673   3094649957   17674244   
736202926163533825 : 9814812   3240396234   26574283   1891266440   185025785   
1032551382790885376 : 380648579   32828872   100248020   1656958189   14885540   
1375786314986004481 : 380648579   32828872   100248020   1656958189   14885540   
274318119 : 9814812   3240396234   26574283   1891266440   185025785   
785781830 : 24259259   346102488   1244013067790364673   3094649957   17674244   
743762631152926722 : 24259259   17674244   1212662806119473159   301374440   15859912   
3015007486 : 227429355   635750362   1230064814589915136   2966462296   947605221416538112   
29414702 : 24259259   346102488   1244013067790364673   3094649957   17674244   
325580157 : 9814812   3240396234   26574283   1891266440   185025785   
1421478031 : 24259259   346102488   1244013067790364673   3094649957   17674244   
432255757 : 56488059   20457